### (11) render ours_v2 results

In [1]:
# guodegang
import cv2, sys, tqdm, os, numpy as np, trimesh
sys.path.append("../")
from utility import show_img_arr, rotate_verts_y
from renderer import render_orthcam
from pred_loader_hsdf_ver_512 import load_dispatcher

def clahe_L(img):
    lab= cv2.cvtColor(img, cv2.COLOR_BGR2LAB) # Converting to LAB channels
    l, a, b = cv2.split(lab) # Splitting the LAB channels
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(5,5)) # Applying CLAHE to L-channel
    cl = clahe.apply(l)
    limg = cv2.merge((cl,a,b)) # Merge channels
    final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR) # Converting back to RGB channels
    return final

def normalize_v3(arr):
    ''' Normalize a numpy array of 3 component vectors shape=(n,3) '''
    lens = np.sqrt(arr[:, 0] ** 2 + arr[:, 1] ** 2 + arr[:, 2] ** 2)
    eps = 0.00000001
    lens[lens < eps] = eps
    arr[:, 0] /= lens
    arr[:, 1] /= lens
    arr[:, 2] /= lens
    return arr

def compute_normal(vertices, faces):
    # Create a zeroed array with the same type and shape as our vertices i.e., per vertex normal
    norm = np.zeros(vertices.shape, dtype=vertices.dtype)
    # Create an indexed view into the vertex array using the array of three indices for triangles
    tris = vertices[faces]
    # Calculate the normal for all the triangles, by taking the cross product of the vectors v1-v0, and v2-v0 in each triangle
    n = np.cross(tris[::, 1] - tris[::, 0], tris[::, 2] - tris[::, 0])
    # n is now an array of normals per triangle. The length of each normal is dependent the vertices,
    # we need to normalize these, so that our next step weights each normal equally.
    normalize_v3(n)
    # now we have a normalized array of normals, one per triangle, i.e., per triangle normals.
    # But instead of one per triangle (i.e., flat shading), we add to each vertex in that triangle,
    # the triangles' normal. Multiple triangles would then contribute to every vertex, so we need to normalize again afterwards.
    # The cool part, we can actually add the normals through an indexed view of our (zeroed) per vertex normal array
    norm[faces[:, 0]] += n
    norm[faces[:, 1]] += n
    norm[faces[:, 2]] += n
    normalize_v3(norm)
    
    return norm

clip_name_list = ['baijia', 'guodegang', 'persuade', 'trump', 'obama']
method_name = "hsdf_merge"

for clip_name in clip_name_list:

    fn_list = sorted(os.listdir("./clip_%s/norm_frames/" % clip_name))

    with open("./clip_%s/fps.txt" % clip_name, 'r') as f:
        fps = float(f.readlines()[0])

    size = (512*5, 512)

    vidcap = cv2.VideoWriter("./clip_%s/result_v2.avi" % clip_name, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, size)

    for fn in tqdm.tqdm(fn_list):

        norm_frame = cv2.imread("./clip_%s/norm_frames/%s" % (clip_name, fn))

        concat_list = [norm_frame]

        # read mesh
        pred_align_mesh = load_dispatcher[method_name]("./clip_%s/results_v2/" % clip_name, fn[:-4], -1)

        # rend mesh
        depth_img, rend_img = render_orthcam(pred_align_mesh, 
                                 xy_mag = (1,1), 
                                 rend_size = (512, 512))
        mask = depth_img==0
        mask3 = np.stack((mask, )*3).transpose((1, 2, 0))
        rend_img = clahe_L(rend_img)

        merge_img = norm_frame.copy()
        merge_img[mask3==0] = rend_img[mask3==0]
        concat_list.append(merge_img)
        os.makedirs("./clip_%s/rend_imgs_v2/grey_front/" % clip_name, exist_ok = True)
        cv2.imwrite("./clip_%s/rend_imgs_v2/grey_front/%s" % (clip_name, fn), merge_img)

        # render normal
        norms = compute_normal(pred_align_mesh.vertices, pred_align_mesh.faces)
        pred_align_mesh.visual = trimesh.visual.ColorVisuals(mesh = pred_align_mesh, 
                                                             vertex_colors = (norms[:,[2,1,0]] + 1) / 2)
        depth_img, rend_img = render_orthcam(pred_align_mesh, 
                                 xy_mag = (1,1), 
                                 rend_size = (512, 512),
                                 flat_shading = True)
        mask = depth_img==0
        mask3 = np.stack((mask, )*3).transpose((1, 2, 0))
        
        merge_img = norm_frame.copy()
        merge_img[mask3==0] = rend_img[:,:,[2,1,0]][mask3==0]
        
        concat_list.append(merge_img)
        os.makedirs("./clip_%s/rend_imgs_v2/norm_front/" % clip_name, exist_ok = True)
        cv2.imwrite("./clip_%s/rend_imgs_v2/norm_front/%s" % (clip_name, fn), merge_img)
        
        pred_align_mesh.vertices = rotate_verts_y(pred_align_mesh.vertices, 60)
        
        depth_img, norm_img = render_orthcam(pred_align_mesh, 
                                 xy_mag = (1,1), 
                                 rend_size = (512, 512),
                                 flat_shading = True)
        
        pred_align_mesh.visual = trimesh.visual.ColorVisuals(mesh = pred_align_mesh)
        
        depth_img, rend_img = render_orthcam(pred_align_mesh, 
                                             xy_mag = (1,1), 
                                             rend_size = (512, 512))
        rend_img = clahe_L(rend_img)
        mask = depth_img==0
        mask3 = np.stack((mask, )*3).transpose((1, 2, 0))
        
        rend_img[mask3!=0] = 0
        norm_img[mask3!=0] = 0
        
        concat_list.append(rend_img)
        concat_list.append(norm_img[:,:,[2,1,0]])
        os.makedirs("./clip_%s/rend_imgs_v2/grey_rot/" % clip_name, exist_ok = True)
        cv2.imwrite("./clip_%s/rend_imgs_v2/grey_rot/%s" % (clip_name, fn), rend_img)
        os.makedirs("./clip_%s/rend_imgs_v2/norm_rot/" % clip_name, exist_ok = True)
        cv2.imwrite("./clip_%s/rend_imgs_v2/norm_rot/%s" % (clip_name, fn), norm_img[:,:,[2,1,0]])
        
        concat_img = np.concatenate(tuple(concat_list), axis = 1)
        #show_img_arr(concat_img, bgr_mode=True)

        vidcap.write(concat_img)

    vidcap.release()

100%|██████████| 839/839 [20:54<00:00,  1.50s/it]


### (12) merge changing and normal v2 (refered to (7))

In [2]:
# guodegang
import cv2, sys, tqdm, os, numpy as np, glob
sys.path.append("../")
from utility import show_img_arr, rotate_verts_y

size = (1920, 1080)
change_frame_num = 10
maintian_frame_num = 30

clip_name_list = ['baijia', 'persuade', 'obama'] #
clip_name2idx = {'obama':0, 'baijia':1, 'theresa':2, 'willemijn':3, 'speak':4, 'persuade':5}

for clip_name in clip_name_list:
    
    clip_idx = clip_name2idx[clip_name]
    fn_list = glob.glob("./clip_%s/norm_frames/*" % clip_name)
    num = len(fn_list)
    
    if clip_name == "obama":
        select_frame = 400
    elif clip_name == "baijia":
        select_frame = 150
    elif clip_name == "persuade":
        select_frame = 152
    
    
    with open("./clip_%s/fps.txt" % clip_name, 'r') as f:
        fps = float(f.readlines()[0]) 
    vidcap = cv2.VideoWriter("./clip_%s/result_v2_compare_%s_highlight.avi" % (clip_name, clip_name), cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, size)
    
    for frame_idx in tqdm.trange(num):
        
        frame = cv2.imread("./video_compare_bg.jpg")
        
        src_img = cv2.imread("./clip_%s/norm_frames/%08d.png" % (clip_name, frame_idx))
        
        front_ours = cv2.imread("./clip_%s/rend_imgs_v2/grey_front/%08d.png" % (clip_name, frame_idx))
        front_deca = cv2.imread("./clip_%s/rend_imgs_DECA/grey_front/%d_%08d.png" % (clip_name, clip_idx, frame_idx))
        front_3ddfav2 = cv2.imread("./clip_%s/rend_imgs_3DDFA_V2/grey_front/%d_%08d.png" % (clip_name, clip_idx, frame_idx))
        front_prnet = cv2.imread("./clip_%s/rend_imgs_PRNet/grey_front/%d_%08d.png" % (clip_name, clip_idx, frame_idx))
        
        rot_ours = cv2.imread("./clip_%s/rend_imgs_v2/grey_rot/%08d.png" % (clip_name, frame_idx))
        rot_deca = cv2.imread("./clip_%s/rend_imgs_DECA/grey_rot/%d_%08d.png" % (clip_name, clip_idx, frame_idx))
        rot_3ddfav2 = cv2.imread("./clip_%s/rend_imgs_3DDFA_V2/grey_rot/%d_%08d.png" % (clip_name, clip_idx, frame_idx))
        rot_prnet = cv2.imread("./clip_%s/rend_imgs_PRNet/grey_rot/%d_%08d.png" % (clip_name, clip_idx, frame_idx))
        
        src_img = cv2.resize(src_img, (360, 360))
        front_ours = cv2.resize(front_ours, (360, 360))
        front_deca = cv2.resize(front_deca, (360, 360))
        front_3ddfav2 = cv2.resize(front_3ddfav2, (360, 360))
        front_prnet = cv2.resize(front_prnet, (360, 360))
        rot_ours = cv2.resize(rot_ours, (360, 360))
        rot_deca = cv2.resize(rot_deca, (360, 360))
        rot_3ddfav2 = cv2.resize(rot_3ddfav2, (360, 360))
        rot_prnet = cv2.resize(rot_prnet, (360, 360))
        
        frame[320:320+360, 20:20+360, :] = src_img
        frame[80:80+360, 410:410+360, :] = front_ours
        frame[80:80+360, 770:770+360, :] = rot_ours
        frame[80:80+360, 1170:1170+360, :] = front_deca
        frame[80:80+360, 1530:1530+360, :] = rot_deca
        
        #frame[600:600+360, 20:20+360, :] = front_deca
        frame[560:560+360, 400:400+360, :] = front_3ddfav2
        frame[560:560+360, 780:780+360, :] = rot_3ddfav2
        frame[560:560+360, 1160:1160+360, :] = front_prnet
        frame[560:560+360, 1540:1540+360, :] = rot_prnet
        
        #show_img_arr(frame, bgr_mode = True)
        
        vidcap.write(frame)
        
        if frame_idx == select_frame:
            for turn_idx in range(2):
                for ratio_idx in range(change_frame_num*2+1):
                    frame = cv2.imread("./video_compare_bg_pause.jpg")

                    src_img = cv2.imread("./clip_%s/norm_frames/%08d.png" % (clip_name, frame_idx))

                    front_ours = cv2.imread("./clip_%s/rend_imgs_v2/grey_front/%08d.png" % (clip_name, frame_idx))
                    front_deca = cv2.imread("./clip_%s/rend_imgs_DECA/grey_front/%d_%08d.png" % (clip_name, clip_idx, frame_idx))
                    front_3ddfav2 = cv2.imread("./clip_%s/rend_imgs_3DDFA_V2/grey_front/%d_%08d.png" % (clip_name, clip_idx, frame_idx))
                    front_prnet = cv2.imread("./clip_%s/rend_imgs_PRNet/grey_front/%d_%08d.png" % (clip_name, clip_idx, frame_idx))

                    rot_ours = cv2.imread("./clip_%s/rend_imgs_v2/grey_rot/%08d.png" % (clip_name, frame_idx))
                    rot_deca = cv2.imread("./clip_%s/rend_imgs_DECA/grey_rot/%d_%08d.png" % (clip_name, clip_idx, frame_idx))
                    rot_3ddfav2 = cv2.imread("./clip_%s/rend_imgs_3DDFA_V2/grey_rot/%d_%08d.png" % (clip_name, clip_idx, frame_idx))
                    rot_prnet = cv2.imread("./clip_%s/rend_imgs_PRNet/grey_rot/%d_%08d.png" % (clip_name, clip_idx, frame_idx))

                    src_img = cv2.resize(src_img, (360, 360))
                    front_ours = cv2.resize(front_ours, (360, 360))
                    front_deca = cv2.resize(front_deca, (360, 360))
                    front_3ddfav2 = cv2.resize(front_3ddfav2, (360, 360))
                    front_prnet = cv2.resize(front_prnet, (360, 360))
                    rot_ours = cv2.resize(rot_ours, (360, 360))
                    rot_deca = cv2.resize(rot_deca, (360, 360))
                    rot_3ddfav2 = cv2.resize(rot_3ddfav2, (360, 360))
                    rot_prnet = cv2.resize(rot_prnet, (360, 360))

                    ratio = float(abs(ratio_idx-change_frame_num))/change_frame_num

                    frame[320:320+360, 20:20+360, :] = src_img
                    frame[80:80+360, 410:410+360, :] = (front_ours.astype(np.float32)*ratio + src_img.astype(np.float32)*(1-ratio)).astype(np.uint8)
                    frame[80:80+360, 770:770+360, :] = rot_ours
                    frame[80:80+360, 1170:1170+360, :] = (front_deca.astype(np.float32)*ratio + src_img.astype(np.float32)*(1-ratio)).astype(np.uint8)
                    frame[80:80+360, 1530:1530+360, :] = rot_deca

                    #frame[600:600+360, 20:20+360, :] = front_deca
                    frame[560:560+360, 400:400+360, :] = (front_3ddfav2.astype(np.float32)*ratio + src_img.astype(np.float32)*(1-ratio)).astype(np.uint8)
                    frame[560:560+360, 780:780+360, :] = rot_3ddfav2
                    frame[560:560+360, 1160:1160+360, :] = (front_prnet.astype(np.float32)*ratio + src_img.astype(np.float32)*(1-ratio)).astype(np.uint8)
                    frame[560:560+360, 1540:1540+360, :] = rot_prnet

                    #show_img_arr(frame, bgr_mode = True)
                    if ratio_idx == 0 and turn_idx == 0:
                        for i in range(maintian_frame_num):
                            vidcap.write(frame)
                #     elif ratio_idx == int(change_frame_num*0.5):
                #         for i in range(maintian_frame_num):
                #             vidcap.write(frame)
                    elif ratio_idx == int(change_frame_num*1.0):
                        for i in range(maintian_frame_num):
                            vidcap.write(frame)
                #     elif ratio_idx == int(change_frame_num*1.5):
                #         for i in range(maintian_frame_num):
                #             vidcap.write(frame)
                    elif ratio_idx == int(change_frame_num*2.0):
                        for i in range(maintian_frame_num):
                            vidcap.write(frame)
                    else:
                        vidcap.write(frame)
    
    vidcap.release()

100%|██████████| 839/839 [02:54<00:00,  4.81it/s]


### (13) render other videos (trump, guodegang) v2 (refer to (10))

In [1]:
import cv2, sys, tqdm, os, numpy as np, glob
sys.path.append("../")
from utility import show_img_arr

size = (1920, 1080)

clip_name = "trump"

change_frame_num = 10
maintian_frame_num = 30
fps = 30

vidcap = cv2.VideoWriter("./clip_%s/result2_v2_1080p.avi" % clip_name, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, size)

num = len(os.listdir("./clip_%s/rend_imgs_v2/grey_front/" % clip_name))

for frame_idx in tqdm.trange(num):
    #frame = np.zeros((1080, 1920, 3), dtype=np.uint8)
    frame = cv2.imread("./other_video_bg.jpg")
    
    src_img = cv2.imread("./clip_%s/norm_frames/%08d.png" % (clip_name, frame_idx))
    grey_front = cv2.imread("./clip_%s/rend_imgs_v2/grey_front/%08d.png" % (clip_name, frame_idx))
    norm_front = cv2.imread("./clip_%s/rend_imgs_v2/norm_front/%08d.png" % (clip_name, frame_idx))
    grey_rot = cv2.imread("./clip_%s/rend_imgs_v2/grey_rot/%08d.png" % (clip_name, frame_idx))
    norm_rot = cv2.imread("./clip_%s/rend_imgs_v2/norm_rot/%08d.png" % (clip_name, frame_idx))
    
    grey_rot = grey_rot[:,60:-60,:]
    norm_rot = norm_rot[:,60:-60,:]
    
    merge_img = np.concatenate((src_img, grey_front, norm_front, grey_rot, norm_rot), axis = 1)
    height = round(float(merge_img.shape[0]) * 1920 / float(merge_img.shape[1]))
    merge_img = cv2.resize(merge_img, (1920, height))
    
    pos_x = round((1080 - height) / 2)
    frame[pos_x:pos_x+height,:,:] = merge_img

    vidcap.write(frame)
vidcap.release()

100%|██████████| 551/551 [01:24<00:00,  6.52it/s]


In [2]:
import cv2, sys, tqdm, os, numpy as np, glob
sys.path.append("../")
from utility import show_img_arr

size = (1920, 1080)

clip_name = "guodegang"

change_frame_num = 10
maintian_frame_num = 30
fps = 30

vidcap = cv2.VideoWriter("./clip_%s/result2_v2_1080p.avi" % clip_name, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, size)

num = len(os.listdir("./clip_%s/rend_imgs_v2/grey_front/" % clip_name))

for frame_idx in tqdm.trange(num):
    #frame = np.zeros((1080, 1920, 3), dtype=np.uint8)
    frame = cv2.imread("./other_video_bg.jpg")
    
    src_img = cv2.imread("./clip_%s/norm_frames/%08d.png" % (clip_name, frame_idx))
    grey_front = cv2.imread("./clip_%s/rend_imgs_v2/grey_front/%08d.png" % (clip_name, frame_idx))
    norm_front = cv2.imread("./clip_%s/rend_imgs_v2/norm_front/%08d.png" % (clip_name, frame_idx))
    grey_rot = cv2.imread("./clip_%s/rend_imgs_v2/grey_rot/%08d.png" % (clip_name, frame_idx))
    norm_rot = cv2.imread("./clip_%s/rend_imgs_v2/norm_rot/%08d.png" % (clip_name, frame_idx))
    
    grey_rot = grey_rot[:,60:-60,:]
    norm_rot = norm_rot[:,60:-60,:]
    
    merge_img = np.concatenate((src_img, grey_front, norm_front, grey_rot, norm_rot), axis = 1)
    height = round(float(merge_img.shape[0]) * 1920 / float(merge_img.shape[1]))
    merge_img = cv2.resize(merge_img, (1920, height))
    
    pos_x = round((1080 - height) / 2)
    frame[pos_x:pos_x+height,:,:] = merge_img

    vidcap.write(frame)
vidcap.release()

100%|██████████| 575/575 [01:27<00:00,  6.59it/s]


### (14) make a input_img array for comparison in the last part of video

In [5]:
import numpy as np, cv2, os

fn_list = os.listdir("../qual_eva_wild/input_img_array/")
fn_list = [fn for fn in fn_list if fn[-8:]!="_arr.npy"]

for fn in fn_list:
    src_img = cv2.imread("../qual_eva_wild/input_img_array/" + fn)
    src_img = cv2.resize(src_img, (256, 256))
    img_arr = np.concatenate((src_img, )*12, axis = 1)
    cv2.imwrite("../qual_eva_wild/input_img_array/%s_arr.png" % fn[:-4], img_arr)

### (15) when rearranging code, remake video
after acceptance

In [3]:
# baijia
import cv2, glob, tqdm

fps = 25
fn_list = glob.glob("/media/hao/Document/Code/3dface_pred/hsdf_pred_test/video_wild/clip_baijia/norm_frames/*.png")
fn_list = sorted(fn_list)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter("/media/hao/Document/Code/3dface_pred/hsdf_pred_test/video_wild/clip_baijia/norm_video.avi", fourcc, fps, (512, 512), True)

for fn in tqdm.tqdm(fn_list):
    frame = cv2.imread(fn)
    out.write(frame)

out.release()

100%|██████████| 625/625 [00:06<00:00, 96.09it/s] 


In [4]:
# guodegang
import cv2, glob, tqdm

fps = 25
fn_list = glob.glob("/media/hao/Document/Code/3dface_pred/hsdf_pred_test/video_wild/clip_guodegang/norm_frames/*.png")
fn_list = sorted(fn_list)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter("/media/hao/Document/Code/3dface_pred/hsdf_pred_test/video_wild/clip_guodegang/norm_video.avi", fourcc, fps, (512, 512), True)

for fn in tqdm.tqdm(fn_list):
    frame = cv2.imread(fn)
    out.write(frame)

out.release()

100%|██████████| 575/575 [00:08<00:00, 69.26it/s]


In [5]:
# obama
import cv2, glob, tqdm

fps = 29.970030
fn_list = glob.glob("/media/hao/Document/Code/3dface_pred/hsdf_pred_test/video_wild/clip_obama/norm_frames/*.png")
fn_list = sorted(fn_list)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter("/media/hao/Document/Code/3dface_pred/hsdf_pred_test/video_wild/clip_obama/norm_video.avi", fourcc, fps, (512, 512), True)

for fn in tqdm.tqdm(fn_list):
    frame = cv2.imread(fn)
    out.write(frame)

out.release()

100%|██████████| 839/839 [00:07<00:00, 110.53it/s]


In [6]:
# Trump
import cv2, glob, tqdm

fps = 23.976024
fn_list = glob.glob("/media/hao/Document/Code/3dface_pred/hsdf_pred_test/video_wild/clip_trump/norm_frames/*.png")
fn_list = sorted(fn_list)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter("/media/hao/Document/Code/3dface_pred/hsdf_pred_test/video_wild/clip_trump/norm_video.avi", fourcc, fps, (512, 512), True)

for fn in tqdm.tqdm(fn_list):
    frame = cv2.imread(fn)
    out.write(frame)

out.release()

100%|██████████| 551/551 [00:10<00:00, 54.41it/s]


In [7]:
# Persuade
import cv2, glob, tqdm

fps = 29.758001
fn_list = glob.glob("/media/hao/Document/Code/3dface_pred/hsdf_pred_test/video_wild/clip_persuade/norm_frames/*.png")
fn_list = sorted(fn_list)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter("/media/hao/Document/Code/3dface_pred/hsdf_pred_test/video_wild/clip_persuade/norm_video.avi", fourcc, fps, (512, 512), True)

for fn in tqdm.tqdm(fn_list):
    frame = cv2.imread(fn)
    out.write(frame)

out.release()

100%|██████████| 684/684 [00:11<00:00, 59.27it/s]


In [9]:
(512, 512) == (52, 512)

False